## Introduction:  Project 11
This notebook creates vacation brochures for 3 destinations across the Universe based on estimated flight distance & time from Earth, temperature, and atmosphere. We will have a list of 8 possible stars and planets you can visit while utilizing LLM API calls to generate 2 activites with date, time, and a 2 sentence brief description. Each brocherure contains the destination name, description of destination that is stored as an attribute of the SpaceDestination class, specs (distance, travel time, etc), and a cheap and expensive version of activities as well as a cost estimate.


In [38]:
# pip installs and dependencies (not all of them yet added)

%pip install openai anthropic python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [39]:
#setup and imports (add below, not done)
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variable
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client with the API key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

#claude api
import os
from anthropic import Anthropic

claude_api_key = os.getenv("CLAUDE_API_KEY")
claude_client = Anthropic(api_key=claude_api_key) if claude_api_key else None


### Class Initialization for SpaceDestination and TourPackage
---
SpaceDestination
- Name: string
- Distance: float (furtherest astronomical unit from Earth)
- Cost: $100 * (astronomical units)
- Description: short paragraph

Activity
- generated by LLM API call

TourPackage
- Destination: SpaceDestination object
- Tier: VIP or Budget
- Activity 1: Activity object
- Activitiy 2: Activity object

In [2]:
class SpaceDestination:
    def __init__(self, name, distance, description):
        self.name = name
        self.distance = distance
        self.cost = round(100 * float(distance), 2)
        self.description = description

    def __str__(self):
        return f"{self.name}\n\nBase Cost:{self.cost}\tDistance:{self.distance}\n{self.description}"


class Activity:
    def __init__(self, name, date, time, cost, description):
        self.name = name
        self.date = date
        self.time = time
        self.cost = cost
        self.description = description
    def __str__(self):
        return f"{self.name}\n\nDate & Time: {self.date}\t{self.time}\t\tCost:{self.cost}\n{self.description}"

class TourPackage:
    def __init__(self, destination, tier, activity1, activity2, activity3, activity4):
        self.dest = destination
        self.tier1 = 'Budget'
        self.act1 = activity1
        self.act2 = activity2
        self.tier2 = 'VIP'
        self.act3 = activity3
        self.act4 = activity4
    
    def __str__(self):
        return (f"Tour Package Tier: {self.tier1}\n"
        f"Activity 1: {self.act1}"
        f"\n\n\nActivity 2: {self.act2}"
        
        f"Tour Package Tier: {self.tier2}\n" 
        f"Activity 1: {self.act3}" 
        f"\n\n\nActivity 2: {self.act4}")

In [ ]:
#List of 8 chooseable destinations
locations = []
locations.append(SpaceDestination("Mars", 2.5, "Desert-like rocky planet with a tenuous atmosphere."),)
locations.append(SpaceDestination("Jupiter", 6.2, "Gas giant known for its colorful, swirling cloud bands and the Great Red Spot"),)
locations.append(SpaceDestination("Uranus", 20.1, "Cold, blue-green, ice giant"),)
locations.append(SpaceDestination("Pluto", 50, "Dwarf planet in the Kuiper Belt featuring varied terrain"),)
locations.append(SpaceDestination("Mercury", 1.45, "Features a rocky surface, a large iron core, and extreme temperature variations"),)
locations.append(SpaceDestination("Saturn", 10.07, "Gas giant with a prominent ring system made of ice and dust and massive gaseous atmosphere"),)
locations.append(SpaceDestination("Ceres", 3.1, "Dwarf planet known for its high water ice content,"),)
locations.append(SpaceDestination("Orcus", 48.91, "Dwarf planet in the Kuiper Belt rich in crystalline water ice"),)

# List of indexes in locations that you are adding TourPackage to
selected_locations = []
count = 3

# User selects 3 locations to make tour packages for
while(count > 0):
    choice = input("Choose a location between 1-8: ")
    selected_locations.append(choice - 1)
    count-= 1



In [42]:
#Calculations (NumPy arrays for distances/times/costs) (add below)
# since I think we have the calculations done in the classes already, we might now need this. was numpy a requirment of the project?

In [ ]:
# Visualizations (Matplotlib)
# - Works with an existing `destinations` list of SpaceDestination objects if present
# - Falls back to a small sample set so the plots always render

import math
import numpy as np
import matplotlib.pyplot as plt


def _fallback_destinations():
    try:
        # Use the class if already defined
        SpaceDestination  # type: ignore[name-defined]
    except NameError:
        # Minimal shim to allow plotting fallback data if class not yet defined
        class SpaceDestination:  # type: ignore[no-redef]
            def __init__(self, name, distance, cost, description=""):
                self.name = name
                self.distance = distance  # kilometers
                self.cost = cost          # USD
                self.description = description
    # Approximate mean distances from Earth in kilometers (varies with orbits)
    return [
        SpaceDestination("Moon", 384_400, 150_000, "Earth's Moon"),
        SpaceDestination("Mars", 225_000_000, 500_000, "The Red Planet"),
        SpaceDestination("Europa", 628_000_000, 700_000, "Jupiter's moon"),
        SpaceDestination("Titan", 1_400_000_000, 800_000, "Saturn's moon"),
        SpaceDestination("Alpha Centauri (sys)", 4.37 * 9.461e12, 1_200_000_000, "Nearest stellar system"),
    ]


def _get_destinations_for_plot():
    # Prefer a variable named `destinations` (list of SpaceDestination)
    try:
        return [d for d in destinations if hasattr(d, "name") and hasattr(d, "distance") and hasattr(d, "cost")]  # type: ignore[name-defined]
    except NameError:
        pass
    # Or try a dict/iterable named `destinations_db`
    try:
        db = destinations_db  # type: ignore[name-defined]
        records = []
        if isinstance(db, dict):
            for k, v in db.items():
                name = v.get("name", k)
                distance = v.get("distance", np.nan)
                cost = v.get("cost", np.nan)
                desc = v.get("description", "")
                records.append((name, distance, cost, desc))
        else:
            for v in db:
                name = v.get("name") if isinstance(v, dict) else getattr(v, "name", None)
                distance = v.get("distance") if isinstance(v, dict) else getattr(v, "distance", np.nan)
                cost = v.get("cost") if isinstance(v, dict) else getattr(v, "cost", np.nan)
                desc = v.get("description") if isinstance(v, dict) else getattr(v, "description", "")
                records.append((name, distance, cost, desc))
        # Convert to simple objects with attributes expected below
        class Obj:
            def __init__(self, name, distance, cost, description):
                self.name = name
                self.distance = distance
                self.cost = cost
                self.description = description
        return [Obj(n, d, c, s) for (n, d, c, s) in records if n is not None]
    except NameError:
        pass
    # Fallback sample
    return _fallback_destinations()


def plot_destination_visuals(items=None):
    data = _get_destinations_for_plot() if items is None else items
    # Filter invalid entries
    data = [d for d in data if isinstance(getattr(d, "distance", None), (int, float)) and isinstance(getattr(d, "cost", None), (int, float))]
    if not data:
        print("No destination data available to plot.")
        return

    names = [d.name for d in data]
    distances_km = np.array([max(float(d.distance), 1.0) for d in data], dtype=float)
    costs_usd = np.array([max(float(d.cost), 1.0) for d in data], dtype=float)

    # Use millions of km for numeric stability/readability; log scale will be used
    dist_mkm = distances_km / 1e6

    # Figure layout
    fig, axes = plt.subplots(1, 2, figsize=(14, 5), constrained_layout=True)

    # 1) Distance chart (log scale)
    ax0 = axes[0]
    bars = ax0.barh(names, dist_mkm, color="#4e79a7")
    ax0.set_xscale('log')
    ax0.set_xlabel("Distance (million km, log scale)")
    ax0.set_title("Destination Distances from Earth")
    ax0.grid(True, which='both', axis='x', linestyle='--', alpha=0.3)

    # Annotate bars with values
    for bar, val in zip(bars, dist_mkm):
        if np.isfinite(val):
            ax0.text(val * 1.05, bar.get_y() + bar.get_height() / 2, f"{val:,.2f}", va='center', ha='left', fontsize=9)

    # 2) Cost vs Distance scatter (log-log)
    ax1 = axes[1]
    sc = ax1.scatter(dist_mkm, costs_usd / 1e3, c=dist_mkm, cmap='viridis', s=80, edgecolor='black', alpha=0.85)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlabel("Distance (million km, log scale)")
    ax1.set_ylabel("Estimated Package Cost (thousand USD, log scale)")
    ax1.set_title("Cost vs Distance")
    ax1.grid(True, which='both', linestyle='--', alpha=0.3)

    # Add labels next to points
    for x, y, label in zip(dist_mkm, costs_usd / 1e3, names):
        if np.isfinite(x) and np.isfinite(y):
            ax1.annotate(label, (x, y), textcoords="offset points", xytext=(6, 4), ha='left', fontsize=9)

    # Colorbar for distance coloring
    cbar = fig.colorbar(sc, ax=ax1)
    cbar.set_label("Distance (million km)")

    fig.suptitle("Space Tourism Destinations — Distances and Costs", fontsize=14, fontweight='bold')
    plt.show()


# Execute the visualization with whatever data is available
plot_destination_visuals()


In [44]:
#Brochure formatting: make the brochure look polished and put together (possible with color and different text types or sizes)
#note(delete note later): currently, the brochure is printed at the bottom of the LLM integration cell, so that might have to be changed

In [ ]:
#LLM integration
#LLM picks 3 of our 8 destinations and generates 3 activities each based on user input(keywords and constraints, goals)

#this function prompts the LLM to pick 3 of the 8 destinations based on user goals
def llm_pick_destinations(user_goals, available_destinations):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": f"From these destinations: {[d.name for d in available_destinations]}, pick 3 that best match these goals: '{user_goals}'. Reply with just the 3 names separated by commas."}],
            max_tokens=40,
        )
        return response.choices[0].message.content.strip()
    except:
        return "Mars, Jupiter, Saturn"

#prompts the LLM to generate activities based on user goals
def llm_generate_activities(destination, user_goals, distance, cost):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": f"Generate 3 space activities for {destination}. Goals: {user_goals}. Budget: ${cost}. Respond ONLY with these 3 lines, nothing else:\n\nActivity1: [name] - [description]\nActivity2: [name] - [description]\nActivity3: [name] - [description]"}],
            max_tokens=150, # high enough to allow LLM to provide complete sentence within token limit. if limit exceeded, may output incomplete sentence, but fixing this results in slow code due to repetetive LLM calls.
            temperature=0.1,  # Very low temperature for consistency
        )
        return response.choices[0].message.content.strip()
    except:
        return f"Activity1: Surface Walk - Explore {destination} terrain\nActivity2: Photo Session - Capture {destination} views\nActivity3: Orbital Tour - See {destination} from space"
        
# user input for goals/constraints
print("Welcome to Space Tourism!")
print("What are your goals/constraints? (e.g., 'budget-friendly', 'luxury experience', 'adventure', 'relaxation')")
#print("Here is your package with some destinations and activities we think you'd enjoy!")
user_goals = input("Your goals: ").strip()

# LLM picks 5 destinations from your 8 options
selected_names = llm_pick_destinations(user_goals, locations)
selected_destinations = [dest for dest in locations if dest.name in selected_names]

# Generate brochures for selected destinations
for dest in selected_destinations:
    # LLM generates 3 activities based on user goals
    activities_text = llm_generate_activities(dest.name, user_goals, dest.distance, dest.cost)

    # Parse activities
    activities = []
    for line in activities_text.split('\n'):
        if 'Activity' in line and ':' in line:
            try:
                name_desc = line.split(':', 1)[1].strip()
                name, desc = name_desc.split(' - ', 1)
                activities.append((name.strip(), desc.strip()))
            except:
                pass

    #fallback if still not 3
    if len(activities) < 3:
        fallbacks = [
            ("Surface Walk", f"Explore {dest.name} terrain"),
            ("Photo Session", f"Capture {dest.name} views"),
            ("Orbital Tour", f"See {dest.name} from space")
        ]
        
    while len(activities) < 3:
        activities.append(fallbacks[len(activities)])

    # Create Activity objects
    act1 = Activity(activities[0][0], "2024-01-15", "10:00", dest.cost * 0.5, activities[0][1])
    act2 = Activity(activities[1][0], "2024-01-16", "14:00", dest.cost * 0.8, activities[1][1])
    act3 = Activity(activities[2][0], "2024-01-17", "09:00", dest.cost * 1.2, activities[2][1])
        
    # Create package
    package = TourPackage(dest, "Budget/VIP", act1, act2, act3, act3)
        
    #print brochure with LLM chosen destinations and activities
    print(f"\n=== {dest.name} Space Tour ===")
    print(f"Distance: {dest.distance} AU | Cost: ${dest.cost}")
    print(f"1. {act1.name} (${act1.cost}): {act1.description}")
    print(f"2. {act2.name} (${act2.cost}): {act2.description}")
    print(f"3. {act3.name} (${act3.cost}): {act3.description}")
    print("-" * 50)

Welcome to Space Tourism!
What are your goals/constraints? (e.g., 'budget-friendly', 'luxury experience', 'adventure', 'relaxation')
LLM response for Pluto:
"Activity1: Pluto Scavenger Hunt - Create a scavenger hunt with themed clues and challenges related to Pluto's features and mythology, engaging the whole family in a fun outdoor adventure.  \nActivity2: Pluto Movie Night - Host a movie night featuring documentaries and films about Pluto and the solar system, complete with themed snacks and cozy blankets for family bonding.  \nActivity3: Pluto Art Workshop - Organ"
---
Parsed: Pluto Scavenger Hunt - Create a scavenger hunt with themed clues and challenges related to Pluto's features and mythology, engaging the whole family in a fun outdoor adventure.
Parsed: Pluto Movie Night - Host a movie night featuring documentaries and films about Pluto and the solar system, complete with themed snacks and cozy blankets for family bonding.
Parsed: Pluto Art Workshop - Organ
Total activities par

results and discussion (add below)

individual contribution statement (add below)